# Posterior Sampling for Gaussian Mixture Model with CRP using Gibbs sampler

Reference: https://pdfs.semanticscholar.org/9ece/0336316d78837076ef048f3d07e953e38072.pdf

In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
# Lets generate some data
X1 = np.random.multivariate_normal([5, 5], np.diag([0.5, 0.5]), size=20)
X2 = np.random.multivariate_normal([8, 8], np.diag([0.5, 0.5]), size=20)
X3 = np.random.multivariate_normal([20, 20], np.diag([0.5, 0.5]), size=10)

X = np.vstack([X1, X2, X3])

X

array([[  6.38718904,   6.26744299],
       [  4.91069596,   5.05729021],
       [  5.6358458 ,   3.76841987],
       [  4.69085752,   5.87209214],
       [  4.51370654,   4.57976556],
       [  5.22511027,   3.26420955],
       [  4.30695956,   4.99733247],
       [  5.67752375,   4.59798902],
       [  4.76786938,   5.29183611],
       [  5.68605653,   5.48175125],
       [  4.5712212 ,   5.86214207],
       [  4.37396308,   4.22177446],
       [  5.48148715,   5.54768881],
       [  6.22246802,   5.58785939],
       [  4.59196756,   4.83510177],
       [  4.77665056,   5.37374645],
       [  4.41072153,   4.11089838],
       [  4.37349841,   3.96235133],
       [  4.98377582,   5.64058177],
       [  4.94854768,   6.42038406],
       [  7.29714298,   7.02506548],
       [  8.43446672,   7.97241902],
       [  7.88298324,   7.55896888],
       [  8.90127699,   8.46433482],
       [  9.37849218,   8.01302645],
       [  8.69586662,   7.59193842],
       [  7.73618367,   8.13888114],
 

In [4]:
N, D = X.shape

In [5]:
# GMM paramters
mus = [] # This lists a 2x1 vector (mean vector of each gaussian)
sigma = np.eye(D)
precision = np.linalg.inv(sigma)
zs = np.zeros([N], dtype=int)
C = [] # Cluster, bu=inary matrix of KxM
Ns = [] # Count of each cluster

In [6]:
sigma

array([[ 1.,  0.],
       [ 0.,  1.]])

In [7]:
precision

array([[ 1.,  0.],
       [ 0.,  1.]])

In [8]:
zs

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [9]:
# CRP prior
alpha = 200

In [10]:
# Base distribution prior: N(mutheta, precisiontheta)
mu_theta = np.ones(D)
sigma_theta = np.eye(D)
precision_theta = np.linalg.inv(np.eye(D))
G_theta = stats.multivariate_normal(mean=mu_theta, cov=np.eye(D))

In [11]:
# Initiate the ONE cluster
C.append(np.ones(N, dtype=int))
zs[:] = 0
Ns.append(N)
mus.append(G_theta.rvs())

K = 1
mvn = stats.multivariate_normal

In [15]:
# Gibbs sampler
for it in range(20):
    # here we sample from full conditional of assignment from CRP prior
    # z ~ GEM(alpha)
    # Now, for each data point, draw a cluster assignment
    for i in range(N):
         # Remove assignment from cluster
        # ------------------------------

        zi = zs[i]
        C[zi][i] = 0
        Ns[zi] -= 1

        # If empty, remove cluster
        if Ns[zi] == 0:
            # Fix indices
            zs[zs > zi] -= 1

            # Delete cluster
            del C[zi]
            del Ns[zi]
            del mus[zi]

            # Decrement cluster count
            K -= 1

        # Draw new assignment zi weighted by CRP prior
        # --------------------------------------------

        probs = np.zeros(K + 1)
        zs_minus_i = zs[np.arange(len(zs)) != i]

        # Probs of joining existing cluster
        for k in range(K):
            nk_minus = zs_minus_i[zs_minus_i == k].shape[0]
            crp = nk_minus / (N + alpha - 1)
            probs[k] = crp * mvn.pdf(X[i], mus[k], sigma)

        # Prob of creating new cluster
        crp = alpha / (N + alpha - 1)
        lik = mvn.pdf(X[i], mu_theta, sigma_theta + sigma)  # marginal dist. of x
        probs[K] = crp*lik

        # Normalize
        probs /= np.sum(probs)

        # Draw new assignment for i
        z = np.random.multinomial(n=1, pvals=probs).argmax()

        # Update assignment trackers
        if z == K:
            C.append(np.zeros(N, dtype=int))
            Ns.append(0)
            mus.append(G_theta.rvs())
            K += 1

        zs[i] = z
        C[z][i] = 1
        Ns[z] += 1

    # -------------------------------------------------
    # Sample from full conditional of cluster parameter
    # -------------------------------------------------

    # Assume fixed covariance => posterior is Normal
    # mu ~ N(mu, sigma)
    for k in range(K):
        Xk = X[zs == k]
        Ns[k] = Xk.shape[0]

        # Covariance of posterior
        lambda_post = precision_theta + Ns[k]*precision
        cov_post = np.linalg.inv(lambda_post)

        # Mean of posterior
        left = cov_post
        right = precision_theta @ mu_theta + Ns[k]*precision @ np.mean(Xk, axis=0)
        mus_post = left @ right

        # Draw new mean sample from posterior
        mus[k] = mvn.rvs(mus_post, cov_post)

##  Even though we only initialize with one cluster, the result should be:

 **Expected output:**
``` python 
20 data in cluster-0, mean: [ 5  5 ]
20 data in cluster-1, mean: [ 8  8 ]
10 data in cluster-2, mean: [ 20  20 ]
```

 Note: cluster label is exchangeable

In [17]:
for k in range(K):
    print('{} data in cluster-{}, mean: {}'.format(Ns[k], k, mus[k]))

19 data in cluster-0, mean: [ 4.5457113   4.93233594]
21 data in cluster-1, mean: [ 8.25548463  7.50082634]
10 data in cluster-2, mean: [ 18.60459256  18.30911234]
